In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import time
import os
warnings.filterwarnings('ignore')

In [2]:
import wavio
import librosa

In [4]:
filename_list = ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']

In [5]:
y=[]
X_train = []
for i in range(30):
    filePath = "D:\\ccf2020\\%s" % filename_list[i]
    fl = os.listdir(filePath)
    print(len(fl))
    for j in range(len(fl)):
        wavpath = filePath + '\\' + fl[j]
        sig,sr = librosa.load(wavpath)
        y.append(int(i))
        #print(len(sig))
        temp = []
        #过零率
        a = librosa.feature.zero_crossing_rate(sig,sr)
        b = np.mean(a)
        temp.append(b)
        #光谱质心
        a = librosa.feature.spectral_centroid(sig,sr=sr)[0]
        b = np.mean(a)
        temp.append(b)
        #MFCC
        a = librosa.feature.mfcc(sig,sr,n_mfcc=40)
        b = np.mean(a,axis = 1)
        for k in range(len(b)):
            temp.append(b[k])
        #色度频谱
        a = librosa.feature.chroma_stft(sig,sr)
        b = np.mean(a,axis = 1)
        for k in range(len(b)):
            temp.append(b[k])
        #谱对比度
        a = librosa.feature.spectral_contrast(sig,sr)
        b = np.mean(a,axis = 1)
        for k in range(len(b)):
            temp.append(b[k]) 
        #频谱带宽
        a = librosa.feature.spectral_bandwidth(sig,sr)
        b = np.mean(a,axis=1)
        temp.append(b)
        #tonnetz
        a = librosa.feature.tonnetz(sig,sr)
        b = np.mean(a,axis = 1)
        for k in range(len(b)):
            temp.append(b[k])
        if j < 2:
            print(len(temp))
        X_train.append(temp)
    print('*****%s*****'%i)

1537
68
68
*****0*****
1573
68
68
*****1*****
1567
68
68
*****2*****
1566
68
68
*****3*****
2106
68
68
*****4*****
2095
68
68
*****5*****
2086
68
68
*****6*****
2119
68
68
*****7*****
2121
68
68
*****8*****
1562
68
68
*****9*****
1600
68
68
*****10*****
2086
68
68
*****11*****
1584
68
68
*****12*****
2105
68
68
*****13*****
2123
68
68
*****14*****
2095
68
68
*****15*****
2121
68
68
*****16*****
2122
68
68
*****17*****
2108
68
68
*****18*****
2138
68
68
*****19*****
1548
68
68
*****20*****
2125
68
68
*****21*****
2131
68
68
*****22*****
2089
68
68
*****23*****
1540
68
68
*****24*****
2109
68
68
*****25*****
2103
68
68
*****26*****
1580
68
68
*****27*****
2121
68
68
*****28*****
2126
68
68
*****29*****


In [14]:
X_test = []
filePath = "D:\\ccf2020\\test"
fl = os.listdir(filePath)
print(len(fl))

for j in range(len(fl)):
    wavpath = filePath + '\\' + fl[j]
    sig,sr = librosa.load(wavpath)
    #print(len(sig))
    temp = []
    #过零率
    a = librosa.feature.zero_crossing_rate(sig,sr)
    b = np.mean(a)
    temp.append(b)
    #光谱质心
    a = librosa.feature.spectral_centroid(sig,sr=sr)[0]
    b = np.mean(a)
    temp.append(b)
    #MFCC
    a = librosa.feature.mfcc(sig,sr,n_mfcc=40)
    b = np.mean(a,axis = 1)
    for k in range(len(b)):
        temp.append(b[k])
    #色度频谱
    a = librosa.feature.chroma_stft(sig,sr)
    b = np.mean(a,axis = 1)
    for k in range(len(b)):
        temp.append(b[k])
    #谱对比度
    a = librosa.feature.spectral_contrast(sig,sr)
    b = np.mean(a,axis = 1)
    for k in range(len(b)):
        temp.append(b[k]) 
    #频谱带宽
    a = librosa.feature.spectral_bandwidth(sig,sr)
    b = np.mean(a,axis=1)
    temp.append(b)
    #tonnetz
    a = librosa.feature.tonnetz(sig,sr)
    b = np.mean(a,axis = 1)
    for k in range(len(b)):
        temp.append(b[k])

    if j<=5:
        print(len(temp))
    X_test.append(temp)

6835
68
68
68
68
68
68


In [8]:
X_train_c = np.array(X_train)

In [9]:
print(X_train_c.shape)

(57886, 68)


In [15]:
X_test_c = np.array(X_test)

In [16]:
print(X_test_c.shape)

(6835, 68)


In [12]:
y1_train_c = np.array(y)

In [13]:
print(y1_train_c.shape)

(57886,)


In [7]:
X_test = []
filePath = "D:\\ccf2020\\test"
fl = os.listdir(filePath)
print(fl[0:10])

['003gtit8kw.wav', '006irl4pgx.wav', '007sh75o5w.wav', '009k6j5dbw.wav', '009lyahcx8.wav', '00ajqu80u0.wav', '00xs1x0xu4.wav', '00zipjvjsy.wav', '01154zpb2g.wav', '016hhb0ofw.wav']


In [24]:
nfold = 5
kf = KFold(n_splits=nfold, shuffle=True, random_state=2020)
oof = np.zeros((len(X_train_c), ))
prediction1 = np.zeros((len(X_test_c),30 ))

In [25]:
ITERATIONS = 100000
EARLY_STOP = 100
VERBOSE = 500

i = 0
for train_index, valid_index in kf.split(X_train_c, y1_train_c):
    print("\nFold {}".format(i + 1))
    X_train, label_train = X_train_c[train_index],y1_train_c[train_index]
    X_valid, label_valid = X_train_c[valid_index],y1_train_c[valid_index]
    
    params = {'num_leaves': 16, #结果对最终效果影响较大，越大值越好，太大会出现过拟合
              'min_data_in_leaf': 16,
              'objective': 'multiclass', #定义的目标函数
              'is_unbalance' : True,
              'max_depth': 8,
              'learning_rate': 0.08,
              'boosting': 'gbdt',
              'feature_fraction': 0.98,  #提取的特征比率
              'bagging_freq': 4,
              'bagging_fraction': 0.8,
              'lambda_l1': 0.1,             #l1正则
              'lambda_l2': 0.2,     #l2正则
              'num_class':30,
              "random_state": 2020, #随机数种子，可以防止每次运行的结果不一致
              }
    trn_data = lgb.Dataset(X_train, label_train)
    val_data = lgb.Dataset(X_valid, label_valid)
    
    clf = lgb.train(params,
                    trn_data,
                    30000,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=VERBOSE,
                    early_stopping_rounds=EARLY_STOP)

    x1 = clf.predict(X_valid)
    y1 = clf.predict(X_test_c)
    prediction1 += ((y1)) / nfold
    i += 1


Fold 1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024036 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17340
[LightGBM] [Info] Number of data points in the train set: 46308, number of used features: 68
[LightGBM] [Info] Start training from score -3.625865
[LightGBM] [Info] Start training from score -3.618592
[LightGBM] [Info] Start training from score -3.599452
[LightGBM] [Info] Start training from score -3.616983
[LightGBM] [Info] Start training from score -3.298237
[LightGBM] [Info] Start training from score -3.316521
[LightGBM] [Info] Start training from score -3.335752
[LightGBM] [Info] Start training from score -3.313549
[LightGBM] [Info] Start training from score -3.320696
[LightGBM] [Info] Start training from score -3.602617
[LightGBM] [Info] Start training from score -3.579898
[LightGBM] [Info] Start training from score -3.327895
[LightGBM] [Info] Start training from score -3.5744

Training until validation scores don't improve for 100 rounds
[500]	training's multi_logloss: 0.202135	valid_1's multi_logloss: 1.87991
Early stopping, best iteration is:
[656]	training's multi_logloss: 0.103574	valid_1's multi_logloss: 1.87188

Fold 5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022639 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17340
[LightGBM] [Info] Number of data points in the train set: 46309, number of used features: 68
[LightGBM] [Info] Start training from score -3.623456
[LightGBM] [Info] Start training from score -3.595532
[LightGBM] [Info] Start training from score -3.611393
[LightGBM] [Info] Start training from score -3.589258
[LightGBM] [Info] Start training from score -3.333350
[LightGBM] [Info] Start training from score -3.333350
[LightGBM] [Info] Start training from score -3.320120
[LightGBM] [Info] Start training from score -3.304708
[LightGBM] [Info] Star

In [26]:
print(prediction1[0])

[0.01449035 0.00027185 0.00182696 0.00044376 0.04093012 0.00420515
 0.00448687 0.01241846 0.00309413 0.14061745 0.00038459 0.010098
 0.04452591 0.15516719 0.03899894 0.00119805 0.05393239 0.04556806
 0.1515222  0.0102623  0.00158036 0.0010757  0.00773553 0.08058271
 0.00142303 0.08520389 0.00296378 0.00042951 0.02139069 0.06317206]


In [46]:
print(sum(prediction1[0]))

1.0


In [27]:
y_pred=[list(x).index(max(x)) for x in prediction1]

In [82]:
print(y_pred)

[13, 28, 0, 5, 12, 20, 22, 26, 5, 18, 15, 29, 20, 6, 4, 10, 18, 18, 27, 17, 0, 4, 2, 18, 4, 22, 25, 5, 14, 21, 24, 23, 1, 25, 21, 1, 26, 22, 21, 23, 8, 29, 17, 17, 19, 18, 13, 18, 13, 0, 18, 28, 2, 29, 21, 6, 12, 19, 26, 26, 14, 21, 27, 12, 0, 5, 12, 14, 6, 6, 25, 9, 12, 18, 16, 18, 6, 13, 11, 2, 6, 19, 18, 18, 11, 17, 14, 25, 2, 23, 16, 22, 5, 23, 4, 13, 12, 6, 18, 3, 25, 26, 12, 25, 8, 26, 16, 5, 22, 24, 8, 24, 26, 29, 24, 24, 29, 24, 25, 20, 13, 19, 8, 20, 3, 11, 23, 18, 20, 7, 28, 22, 5, 7, 21, 6, 5, 28, 21, 28, 6, 29, 5, 29, 5, 15, 9, 28, 3, 28, 17, 4, 18, 15, 24, 13, 28, 24, 26, 11, 27, 6, 21, 14, 11, 16, 10, 25, 17, 14, 11, 18, 29, 26, 8, 28, 26, 21, 13, 18, 13, 5, 14, 18, 0, 19, 7, 12, 28, 24, 19, 11, 29, 20, 9, 13, 11, 18, 22, 29, 22, 17, 18, 11, 20, 25, 29, 11, 11, 11, 14, 19, 11, 24, 15, 18, 17, 29, 22, 1, 8, 26, 2, 14, 14, 1, 3, 17, 14, 20, 28, 22, 9, 8, 19, 23, 20, 5, 28, 24, 3, 18, 26, 6, 14, 11, 18, 4, 19, 28, 16, 14, 6, 13, 9, 20, 15, 21, 7, 25, 7, 5, 23, 23, 2, 15, 26,

In [29]:
sub = pd.read_csv('submission.csv')

In [30]:
cnt = 0

In [31]:
result = [0 for i in range(6835)]
for i in range(6835):
    ss = sub.iloc[i]['file_name']
    for j in range(6835):
        if fl[j] == ss:
            result[i] = y_pred[j]
            cnt = cnt+1
print(cnt)

6835


In [32]:
print(result[0:10])

[28, 22, 6, 22, 25, 22, 14, 16, 26, 26]


In [33]:
result1 = []
for i in range(len(result)):
    result1.append(filename_list[result[i]])

In [34]:
print(result1[0:10])

['yes', 'stop', 'five', 'stop', 'two', 'stop', 'no', 'on', 'up', 'up']


In [35]:
df = pd.DataFrame({'file_name':sub['file_name'],'label':result1})
now = time.strftime("%Y%m%d_%H%M%S",time.localtime(time.time())) 
fname="submit_"+now+r".csv"    
df.to_csv(fname, index=False)

In [19]:
import pickle

In [20]:
fileHandle = open ( 'traindata_v2.txt', 'wb+' ) 
pickle.dump(X_train_c, fileHandle) 
fileHandle.close() 

In [21]:
fileHandle = open ( 'ydata_v2.txt', 'wb+' ) 
pickle.dump(y1_train_c, fileHandle) 
fileHandle.close() 

In [22]:
fileHandle = open ( 'testdata_v2.txt', 'wb+' ) 
pickle.dump(X_test_c, fileHandle) 
fileHandle.close() 